In [81]:
import pandas as pd
import numpy as np
import requests
import json

In [82]:
apikey = "ad88ad93-b2db-485a-a0c2-82da759612b4" 
baseURL = "https://content.guardianapis.com/search?"
section = "science" 
page = 1

#page through all results using for loop
url_list = []
for page in range(1,566):
    url = baseURL+'section='+section+"&page="+str(page)+"&api-key="+apikey
    url_list += [url]

In [13]:
def getArticleData(url):
    response = requests.get(url)
    data = json.loads(response.content)
    result = data['response']['results']
    return result

In [14]:
result = [] #list that contains all results
for i in url_list:
    result = result+[getArticleData(i)]  

In [32]:
title = []
date = []
for k in result:
    for r in k:
        date += [r['webPublicationDate']]
        title += [r['webTitle']]  

In [34]:
science_df = pd.DataFrame({'Date':date,'Title':title})
science_df

,Date,Title
0,2021-06-07T19:47:44Z,"24,000-year-old organisms found frozen in Sibe..."
1,2021-06-07T17:56:30Z,The tech billionaire space race: who is Jeff B...
2,2021-06-07T17:30:10Z,‘Blacklist’ claim: Australian Research Council...
3,2021-06-07T05:00:01Z,How and when to watch the solar eclipse on Thu...
4,2021-06-06T17:14:41Z,When falling down doesn’t get a laugh | Briefs
...,...,...
5645,2015-10-01T18:10:02Z,Smoked ancestors for a pack of smokes | Letters
5646,2015-10-01T16:31:06Z,If the face fits: science of attraction is bas...
5647,2015-10-01T12:29:15Z,A living thing in two places at once? This qua...
5648,2015-10-01T11:40:16Z,Why no Trident? The benefits of a nuclear apo...


In [37]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.utils import tokenize
from gensim.utils import simple_preprocess
from gensim.corpora.textcorpus import remove_stopwords
#from gensim.summarization import keywords
from gensim.models.ldamodel import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models

In [38]:
import warnings
warnings.filterwarnings("ignore")

In [40]:
tokens = list(tokenize(science_df['Title'][0], lowercase = True))

In [45]:
science_df['terms'] = [remove_stopwords(simple_preprocess(title)) for title in science_df['Title']]

In [46]:
science_df

,Date,Title,terms
0,2021-06-07T19:47:44Z,"24,000-year-old organisms found frozen in Sibe...","[year, old, organisms, frozen, siberia, reprod..."
1,2021-06-07T17:56:30Z,The tech billionaire space race: who is Jeff B...,"[tech, billionaire, space, race, jeff, bezos]"
2,2021-06-07T17:30:10Z,‘Blacklist’ claim: Australian Research Council...,"[blacklist, claim, australian, research, counc..."
3,2021-06-07T05:00:01Z,How and when to watch the solar eclipse on Thu...,"[watch, solar, eclipse, thursday]"
4,2021-06-06T17:14:41Z,When falling down doesn’t get a laugh | Briefs,"[falling, laugh, briefs]"
...,...,...,...
5645,2015-10-01T18:10:02Z,Smoked ancestors for a pack of smokes | Letters,"[smoked, ancestors, pack, smokes, letters]"
5646,2015-10-01T16:31:06Z,If the face fits: science of attraction is bas...,"[face, fits, science, attraction, based, perso..."
5647,2015-10-01T12:29:15Z,A living thing in two places at once? This qua...,"[living, thing, places, quantum, quandary, tes..."
5648,2015-10-01T11:40:16Z,Why no Trident? The benefits of a nuclear apo...,"[trident, benefits, nuclear, apocalypse, dean,..."


In [49]:
vocab = Dictionary(science_df['terms'])
print(vocab.token2id) # this function dorectly goves out the frequency of the vocabs

{'frozen': 0, 'old': 1, 'organisms': 2, 'reproduce': 3, 'siberia': 4, 'year': 5, 'bezos': 6, 'billionaire': 7, 'jeff': 8, 'race': 9, 'space': 10, 'tech': 11, 'admits': 12, 'applicants': 13, 'australian': 14, 'blacklist': 15, 'china': 16, 'claim': 17, 'council': 18, 'links': 19, 'research': 20, 'scanning': 21, 'sensitivities': 22, 'eclipse': 23, 'solar': 24, 'thursday': 25, 'watch': 26, 'briefs': 27, 'falling': 28, 'laugh': 29, 'free': 30, 'helen': 31, 'murray': 32, 'obituary': 33, 'avocados': 34, 'heads': 35, 'iss': 36, 'rocket': 37, 'spacex': 38, 'squid': 39, 'toothpaste': 40, 'breast': 41, 'cancer': 42, 'drug': 43, 'help': 44, 'hereditary': 45, 'survive': 46, 'women': 47, 'biddulph': 48, 'fully': 49, 'happier': 50, 'healthier': 51, 'human': 52, 'secret': 53, 'steve': 54, 'supersense': 55, 'age': 56, 'bath': 57, 'end': 58, 'fell': 59, 'know': 60, 'middle': 61, 'missions': 62, 'nasa': 63, 'plans': 64, 'return': 65, 'venus': 66, 'cities': 67, 'crime': 68, 'lockdowns': 69, 'plummets': 70

__Use TF-IDF Model on Titles and Find Most Relevant terms__

In [50]:
# convert corpus to BoW format
corpus = [vocab.doc2bow(terms) for terms in science_df['terms']]  
model = TfidfModel(corpus)# fit a tf-idf model to the corpus
tfidf_doc = model[corpus] # apply model to the first corpus

In [55]:
def get_tfidf (index):
    term_values = [(vocab[e[0]],e[1]) for e in model[corpus[index]] if e[1]>0]
    srt =  sorted(term_values, key=lambda x: x[1],reverse=True)
    return list(map(lambda x: x[0],srt[:5]))

['biddulph', 'supersense', 'fully', 'healthier', 'happier']

__LDA Model of our Corpus__

In [63]:
# create LDA model witg corpus and vocab, and define the topic numbers
lda_model = LdaModel(corpus = corpus, id2word = vocab, num_topics = 20)

In [83]:
# show_topic function returns a list with format of [topic number, topic content]
for topic in lda_model.show_topics(num_topics = 20, num_words = 15):
    print("Topic "+str(topic[0])+"\n"+topic[1]+"\n")

Topic 0
0.044*"solve" + 0.021*"brain" + 0.019*"waves" + 0.016*"cells" + 0.016*"smarter" + 0.013*"lab" + 0.013*"skin" + 0.012*"puzzles" + 0.012*"long" + 0.011*"human" + 0.009*"lies" + 0.009*"drive" + 0.009*"test" + 0.009*"british" + 0.009*"weight"

Topic 1
0.032*"science" + 0.020*"climate" + 0.013*"uk" + 0.013*"change" + 0.012*"new" + 0.012*"lydon" + 0.012*"susannah" + 0.011*"drug" + 0.010*"alcohol" + 0.009*"risks" + 0.009*"rats" + 0.009*"watch" + 0.007*"cure" + 0.007*"open" + 0.007*"good"

Topic 2
0.020*"discovery" + 0.019*"new" + 0.017*"planet" + 0.016*"time" + 0.014*"fossil" + 0.014*"einstein" + 0.013*"nasa" + 0.012*"pluto" + 0.012*"solar" + 0.012*"light" + 0.010*"happened" + 0.010*"images" + 0.010*"probe" + 0.010*"scientists" + 0.008*"discovered"

Topic 3
0.017*"let" + 0.013*"age" + 0.013*"rohn" + 0.012*"darwin" + 0.012*"jenny" + 0.010*"control" + 0.010*"news" + 0.010*"bronze" + 0.010*"flower" + 0.010*"nhs" + 0.010*"earth" + 0.010*"loss" + 0.009*"truth" + 0.009*"man" + 0.008*"scienc

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]

__Get the probability that a certain document belongs to a certain topic__

In [102]:
doc = science_df['Title'][1]
print("doc:\n",doc)
doc_topics = lda_model.get_document_topics(corpus[2] ,minimum_probability=0.3)
print("doc_topics:\n",doc_topics)

for topic in doc_topics:
    terms = [term for term, prob in lda_model.show_topic(topic[0])]
    print(terms)

doc:
 The tech billionaire space race: who is Jeff Bezos up against?
doc_topics:
 [(16, 0.6205992)]
['new', 'british', 'scientists', 'space', 'words', 'dog', 'behaviour', 'iss', 'obituary', 'guide']


Interpetation: the document belongs to the topic shown below it with a probability of x

__Visualize the Spread of Topics__

In [72]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, vocab)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15     0.176491  0.117277       1        1  7.077293
19    -0.114202  0.108995       2        1  5.965988
7     -0.051684 -0.076142       3        1  5.881106
18     0.129019 -0.008756       4        1  5.656931
4     -0.038437  0.072690       5        1  5.331298
6      0.025820 -0.110497       6        1  5.275586
2     -0.039029  0.086534       7        1  5.242112
13    -0.056996  0.044517       8        1  5.049209
9      0.085554  0.015992       9        1  5.040512
10     0.006706 -0.029263      10        1  4.858037
14     0.037855 -0.018755      11        1  4.856687
17     0.108589  0.021261      12        1  4.855611
0      0.030110 -0.124248      13        1  4.840401
11    -0.075023  0.008586      14        1  4.785680
1     -0.072618  0.005427      15        1  4.721448
8      0.001893 -0.091279      16        1  4.608617
5     -0.034185 -0.046874      17        1  4.325072
12    -0.057891 -0.075970      18        1  4.143150
16    -0.043995  0.057921      19        1  3.883035
3     -0.017976  0.042582      20        1  3.602225, topic_info=             Term        Freq       Total Category  logprob  loglift
107         solve  255.000000  255.000000  Default  30.0000  30.0000
33       obituary  232.000000  232.000000  Default  29.0000  29.0000
657       science  358.000000  358.000000  Default  28.0000  28.0000
96        letters  126.000000  126.000000  Default  27.0000  27.0000
63           nasa  130.000000  130.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
379   spectacular    7.366100   16.423116  Topic20  -5.1691   2.5218
361         earth   12.458596  107.486963  Topic20  -4.6436   1.1687
2962       babies    7.622191   19.356044  Topic20  -5.1349   2.3917
657       science   10.729919  358.425892  Topic20  -4.7930  -0.1851
104         years    7.720557   98.136286  Topic20  -5.1221   0.7811

[1161 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1644      4  0.918152  academics
4531      9  0.910664   actually
4529      4  0.946053  addiction
151      13  0.786236      adopt
2136     20  0.892815     adults
...     ...       ...        ...
4293      8  0.877315      yield
4632      9  0.840457  yorkshire
4028      1  0.812581    younger
3260      1  0.201834     zombie
3260     18  0.706419     zombie

[2675 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 20, 8, 19, 5, 7, 3, 14, 10, 11, 15, 18, 1, 12, 2, 9, 6, 13, 17, 4])

__Overall Topic Analysis:__


- One of the most distinctive topic is Topic 1, which contains keywords such as DNA, baby, gender-related terms, medicine, humans. Thus we can understand that this topic may be highly relevant to study in the biological field.


- The second distinctive topic (Topic No.2) contains terms like nasa, mars, mission, space, life...etc. Which indicates that this topic is quite releveant to explorations into outerspace, especially Mars and moon (whci are the terms, too). 


- Some topcs are highly overlapped with each other. Topic 5, 7, 8, 19, 20 gather a cluster. And from the terms thay include, we can observe that their shared traits lies in more general scientific terms such as scientists, science, experiements... etc. So their value are rather low  when distincting topics from articles.


- Overall, in most topics recognized by the model, though we can connect some terms with human knowledge, other terms are compiled by patterns that are out of common senses. This is both a benefit and downside. This means that after calculating the corpus, the machine has found patterns that is difficult for human to find. By utilizing this features, we can efficeintly classify different articles. However, sice we don't know how the model actually differs them, it will be hard for us human to reveal when the algorithm is making a mistake.